### Dependencies and Setup

Load the target image, and a saved simulation to examine

In [ ]:
using Revise
using Paint
using Serialization
using Images, ImageShow
using Plots
using Evolutionary
using BlackBoxOptim
using StaticArrays
using ImageFeatures
using IntervalSets
using ReinforcementLearning
using ReinforcementLearningBase
using ReinforcementLearningZoo
using Combinatorics
using StatsBase

In [ ]:
target = float.(load("../lisa.png"))
hist = Serialization.deserialize("../output/simresult/simlog_100-prims_100000-batch_3-epoch_100-refine.bin")
state = hist.history[end]
nothing

### Find the least costly shape to remove

For each shape, re-color and re-draw without it and compare the loss difference. Find the shape the has least cost to remove.

It seems that this cost tends to be very high, especially for the earlier triangles. Removing them might not be very useful.

In [ ]:
baseloss = imloss(target, state.current, SELoss())

function without(idx)
    shapes = [state.shapes[1:idx-1] ; state.shapes[idx+1:end]]
    # cols = [state.current_colours[1:idx-1] ; state.current_colours[idx+1:end]]
    # bg = state.background
    cols, bg = opaquerecolor(target, shapes, RasterAlgorithmScanline())
    
    img = zero(target) .+ bg
    for k in eachindex(shapes)
        draw!(img, shapes[k], cols[k], RasterAlgorithmScanline())
    end

    imloss(target, img, SELoss()) - baseloss, img
end

minloss, minidx = findmin(map(first, map(without, 1:75)))

### Shape Deltas

For each shape, draw the parts of it that are visible in the final image.

In [ ]:
anim_singles = @animate for i = 1:100
    img = ones(RGB{Float32}, size(target))
    draw!(img, state.shapes[i], state.current_colours[i], RasterAlgorithmScanline())
    for k = i+1:length(state.shapes)
        draw!(img, state.shapes[k], one(RGB{Float32}), RasterAlgorithmScanline())
    end

    plot(img, size=(256,256), title="$i", axis=false, ticks=false)
end
gif(anim_singles, "/tmp/tmp.gif", fps=3, show_msg=false)